In [1]:
import pandas as pd
from datetime import datetime

import os
from docx import Document
from docx2pdf import convert

import warnings
warnings.filterwarnings('ignore')

# remove the doxc
# os.remove("rma_inv_template.docx")

In [2]:
inv_info = pd.read_excel("inv_info_sample.xlsx")
#rma_data['Row_Number'] = rma_data.groupby('Date').cumcount() + 1
#rma_data[rma_data['Inv_No'].isna()]

In [6]:
inv_no_list = inv_info['Inv_No'].unique()

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [7]:
inv_info['Inv_No'].unique()

array(['SAMCO_1', 'SAMCO_2', 'SAMCO_3', 'SAMCO_4'], dtype=object)

In [5]:
def replace_text_in_table(customer, amount, doc_date, inv_no):
    doc = Document('rma_inv_template.docx')
    
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    for run in para.runs:
                        run.text
                        if 'INV_N' in run.text:
                            run.text = run.text.replace('INV_N', inv_no)
                        if 'DOC_DATE' in run.text:
                            run.text = run.text.replace('DOC_DATE', doc_date.strftime('%B %d, %Y'))
                        if 'DUE_DATE' in run.text:
                            run.text = run.text.replace('DUE_DATE', (doc_date + pd.Timedelta(days=30)).strftime('%B %d, %Y'))
                        if 'CUSTOMER' in run.text:
                            run.text = run.text.replace('CUSTOMER', customer)
                        if 'DES' in run.text:
                            run.text = run.text.replace('DES', description)
                        if 'AMOUNT' in run.text:
                            run.text = run.text.replace('AMOUNT', amount)
                                        
    new_doc_path = inv_no + '.docx'
    doc.save(new_doc_path)

In [8]:
for i in rma_data[rma_data['Inv_No'].isna()].index:
    [customer, amount, doc_date, row_num, description] = rma_data.loc[i, ['Customer', 'Amount', 'Date', 'Row_Number', 'Description']]
    amount = '{0:.2f}'.format(amount)
    inv_no = "LURMA" + doc_date.strftime('%Y%m%d') + str(row_num).zfill(3)
    
    # create the pdf invoice
    replace_text_in_table(customer, amount, doc_date, inv_no)
    convert(inv_no + ".docx")
    os.remove(inv_no + ".docx")
    
    rma_data.loc[i, 'inv_no'] = inv_no
    print(inv_no + " generated successfully!")